## Antimicrobial resistance prediction

Given sequence of nucleotides predict if they are likely to be a pathogenic islands for a bacteria or not.

Input: Sequence. 

Output: SequenceID Sequence Subtype Class Subclass Accuracy %  

This is just a test run to develop encoding methods and model architecture. For the actual training, download all the entries and "Mix and match the different subclasses" to have a good coverages. 
https://www.ncbi.nlm.nih.gov/pathogens/refgene/#

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

2025-09-18 23:24:57.304512: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-18 23:24:57.511290: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Pre-processing

In [2]:
from Bio import SeqIO

def fasta_to_dataframe(fasta_file, cols=['RefSeq nucleotide', 'Sequence nucleotide']):
    # Parse the fasta file and extract the records
    records = list(SeqIO.parse(fasta_file, "fasta"))
    
    # Create a dictionary with ID and Sequence
    ids = [record.id.split(":")[0] for record in records]
    sequences = [str(record.seq) for record in records]
    data = {
        cols[0]: ids,
        cols[1]: sequences
    }
    
    # Convert the dictionary to a pandas DataFrame
    return pd.DataFrame(data)

# Example usage
nucleotide_fasta = './complete_dataset/data/nucleotide.fna'
AA_fasta = './complete_dataset/data/proteins.faa'

nucleotide_df = fasta_to_dataframe(nucleotide_fasta)
AA_df = fasta_to_dataframe(AA_fasta, cols=['RefSeq protein', 'Sequence protein'])

print(nucleotide_df.head())
print(AA_df.head())

  RefSeq nucleotide                                Sequence nucleotide
0       NC_000913.3  AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGG...
1       NC_000913.3  AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGG...
2       NC_000913.3  AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGG...
3       NC_000913.3  AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGG...
4       NC_000913.3  AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGG...
   RefSeq protein                                   Sequence protein
0  WP_003703066.1  MLIKSEYKPRMLPKEEQVKKPMTSNGRISFVLMAMAVLFACLIARG...
1  WP_025297907.1  MENTVFDVRHTFQLTSTELRELQTLLAVVYGGDFSPEDWDHALGGL...
2  WP_063839881.1  MKDRSHDDSMAEVCRNTSENHWLKTDYRTLFRLCPDGRIERENDPD...
3  WP_071224044.1  MNHRKGNEPTAAALMELHVLAMFTHDGNMQIRTINEPWPGEELAPR...
4  WP_004918308.1  MGIEYRSLHTSQLTLSEKEALYDLLIEGFEGDFSHDDFAHTLGGMH...


In [3]:
general_df = pd.read_csv('./complete_dataset/AMR_genes.tsv',sep='\t')
general_df = general_df.dropna()
general_df = pd.merge(general_df, nucleotide_df, on='RefSeq nucleotide', how='inner')
general_df = pd.merge(general_df, AA_df, on='RefSeq protein', how='inner')


In [4]:
from lib.encoding import prepare_data

# --------------------------
# Example usage
# --------------------------
# Assuming general_df is your dataframe
max_len = 200
X_padded, y, label_encoder = prepare_data(df=general_df, max_len=max_len)
num_classes = len(label_encoder.classes_)


import joblib
joblib.dump(label_encoder, "label_encoder.pkl")

['label_encoder.pkl']

In [ ]:
from lib.model import ResistanceModel
# Build & train model
trainer = ResistanceModel(max_len=max_len, num_classes=num_classes, model_type="lstm")
history = trainer.train(X_padded, y, epochs=10, batch_size=32, validation_split=0.2)

# Make predictions
preds = trainer.predict(X_padded[:5])
predicted_classes = label_encoder.inverse_transform(preds.argmax(axis=1))
print("Predicted classes:", predicted_classes)

trainer.model.save("./model/resistance_model.h5")


In [ ]:
trainer = ResistanceModel(max_len=max_len, num_classes=num_classes, model_type="lstm")

trainer.model.save("./model/resistance_model.keras")


In [ ]:
from tensorflow.keras.models import load_model
import joblib
from lib.encoding import GeneEncoder

# Load model + label encoder
model = load_model("./model/resistance_model.keras")
label_encoder = joblib.load("./model/label_encoder.pkl")

# Reuse encoder
encoder = GeneEncoder()

# Example new sequences
new_sequences = [
    "MDEKELIERAGGPVTRGR",   # looks like aminoglycoside
    "MAYEFCEIGESNEYIILA"    # looks like beta-lactam
]

# Encode + pad to same max_len used in training
max_len = 200  # ⚠️ must match training
X_new = encoder.encode_protein_batch(new_sequences, max_len=max_len)

# Predict
preds = model.predict(X_new)
predicted_classes = label_encoder.inverse_transform(preds.argmax(axis=1))

print("Predictions:", predicted_classes)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
Predictions: ['LIPOPEPTIDE' 'QUINOLONE']


2025-09-18 23:27:04.528334: E tensorflow/core/util/util.cc:131] oneDNN supports DT_BOOL only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
